In [19]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def load_data(file_path):
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    y = 2 * y - 1  # Convert labels to {1, -1}
    return X, y

def dual_objective(alphas, X, y, C):
    m = len(y)
    term1 = 0.5 * np.sum(alphas * alphas)
    term2 = -np.sum(alphas)
    term3 = 0.5 * np.dot(alphas * y, np.dot(X, X.T) @ (alphas * y))
    return term1 + term2 + C * term3

def dual_constraint(alphas, y):
    return np.sum(alphas * y)

def train_dual_svm(X, y, C):
    m, n = X.shape
    initial_alphas = np.zeros(m)
    
    # Define the constraints
    constraint = {'type': 'eq', 'fun': lambda alphas: dual_constraint(alphas, y)}

    # Optimize the dual objective function
    result = minimize(dual_objective, initial_alphas, args=(X, y, C), constraints=constraint, method='SLSQP')
    
    # Extract the Lagrange multipliers (alphas)
    alphas = result.x
    
    # Calculate the weights
    w = np.dot(alphas * y, X)
    
    # Calculate the bias
    sv_indices = np.where((alphas > 1e-5) & (alphas < C - 1e-5))[0]
    b = np.mean(y[sv_indices] - np.dot(X[sv_indices], w))
    
    return w, b


def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

def calculate_error(predictions, labels):
    misclassified = np.sum(predictions != labels)
    error = misclassified / len(labels)
    return error

train_file_path = "C:\\Users\\bindu\\Downloads\\Dataset-svm\\bank-note\\train.csv"
test_file_path = "C:\\Users\\bindu\\Downloads\\Dataset-svm\\bank-note\\test.csv"
X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Set hyperparameter
C_values = [100/873, 500/873, 700/873]

# Train and evaluate dual SVM for each C value
for C in C_values:
    print(f"\nTraining Dual SVM with C={C}...\n")
    w_dual, b_dual = train_dual_svm(X_train, y_train, C)
    
    print(f"Weight Vector (w_dual) for C={C}: {w_dual}")
    print(f"Bias (b_dual) for C={C}: {b_dual}")

    # Make predictions on the train set
    train_predictions = predict(X_train, w_dual, b_dual)
    train_error = calculate_error(train_predictions, y_train)
    print(f"Train Error (C={C}): {train_error}")

    # Make predictions on the test set
    test_predictions = predict(X_test, w_dual, b_dual)
    test_error = calculate_error(test_predictions, y_test)
    print(f"Test Error (C={C}): {test_error}")



Training Dual SVM with C=0.1145475372279496...

Weight Vector (w_dual) for C=0.1145475372279496: [-2.50258300e+00 -1.36379421e+00 -1.78226257e+00  2.46114918e-03]
Bias (b_dual) for C=0.1145475372279496: 5.43990657642657
Train Error (C=0.1145475372279496): 0.026376146788990827
Test Error (C=0.1145475372279496): 0.024

Training Dual SVM with C=0.572737686139748...

Weight Vector (w_dual) for C=0.572737686139748: [-0.501384   -0.2730924  -0.35727902  0.00075264]
Bias (b_dual) for C=0.572737686139748: 1.0504399510077336
Train Error (C=0.572737686139748): 0.02408256880733945
Test Error (C=0.572737686139748): 0.022

Training Dual SVM with C=0.8018327605956472...

Weight Vector (w_dual) for C=0.8018327605956472: [-0.35818195 -0.19507173 -0.25521849  0.00053823]
Bias (b_dual) for C=0.8018327605956472: 0.7534787228568641
Train Error (C=0.8018327605956472): 0.02408256880733945
Test Error (C=0.8018327605956472): 0.022
